[Manual](https://github.com/pybox2d/pybox2d/wiki/manual)

[Microsoft Visual C++ Compiler for Python 2.7](https://www.microsoft.com/en-us/download/confirmation.aspx?id=44266)

# System Requirements

- python 2.7.6 (python -V)
- Pygame 1.9.3 (pip freeze | grep pygame)
- swig [swig.exe](http://www.swig.org/download.html)

```
sudo apt-get install python
sudo pip install pygame
sudo apt-get install swig
```

1.

```
git clone https://github.com/pybox2d/pybox2d.git
python setup.py build
sudo python setup.py install
```

2.

```
sudo pip install box2d
```

3.

[pybox2d](https://code.google.com/archive/p/pybox2d/downloads)

In [1]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division
from Box2D import *
from pygame.locals import (QUIT, KEYDOWN, K_ESCAPE)
import math
import random
import operator
import collections
import heapq as hq
import numpy as np
import pygame as pg
import matplotlib.path as pth
import matplotlib.pyplot as plt


def rand(polygon):
    border = pth.Path(polygon)
    while True:
        p = (random.uniform(min(x), max(x)), random.uniform(min(y), max(y)))
        if border.contains_point(p):
            return p


def plot():
    plt.plot(x, y, marker='*')
    for m in modules:
        plt.plot([m.position[0]], [m.position[1]], marker='o')
    plt.show()


def world2screen(v):
    world = np.array((max(x) - min(x), max(y) - min(y)))
    screen = np.array(SCREEN)
    scale = min(np.divide(screen, world))
    offset = screen / 2 - scale * np.array((min(x) + max(x), min(y) + max(y))) / 2
    r = scale * np.array(v) + offset
    r[1] = SCREEN[1] - r[1]
    return r


# a polygon area calculator (CCW)
# Shoelace formula
def area(x, y): return (np.roll(x, 1).dot(y) - np.roll(y, 1).dot(x)) / 2


def distance2line(p, l): return 2 * area(*zip(p, *l)) / np.linalg.norm(np.subtract(*l))


def body2worldvs(b): return map(
    lambda v: b.transform * v, b.fixtures[0].shape.vertices)


def vs2xy(vs, x, y): return sorted(map(lambda v: [distance2line(
    v, y), distance2line(v, x)], vs), key=operator.itemgetter(0))


def x2y(x, xy): return np.array(x).dot(np.linalg.lstsq(
    *zip(*map(lambda (x, y): [[1, x], y], xy)))[0])


def surface_distance(vs, es, x, y):
    surf_dist = []
    for v in vs:
        for e in es:
            xy = vs2xy(e, (v, v + x), (v, v + y))
            if xy[0][0] < 0 < xy[1][0]:
                surf_dist.append(x2y([1, 0], xy))
    return surf_dist

In [2]:
N = 3
POLYGON = 'tr1_1.csv'
lam = 0
TARGET_FPS = 24
TIME_STEP = 1 / TARGET_FPS
SCREEN = (640, 480)
random.seed(0)

In [3]:
# read the polygon from .csv file with normalizing and scaling
polygon = np.array(
    map(lambda p: map(eval, p.split()), open(POLYGON))) / 100 #* 1.5
polygon = (polygon - polygon.mean(axis=0))
polygon = polygon[::-1]
x, y = zip(*polygon)
polygonArea = area(x, y)
print polygonArea
# This defines a triangle in CCW order.

5.17306601162


In [4]:
module = np.array(
    map(lambda a: [math.cos(a), math.sin(a)], 2 * math.pi / N * np.arange(N)))
moduleArea = area(*zip(*module))
print moduleArea
B_P = int(polygonArea / moduleArea)

1.29903810568


In [5]:
world = b2World(gravity=(0, 0), doSleep=False)
borders = map(lambda v: world.CreateStaticBody(shapes=b2EdgeShape(vertices=(
    v - np.mean(v, axis=0)).tolist()), position=np.mean(v, axis=0).tolist()), zip(np.roll(polygon, 2), polygon))
modules = map(lambda _: world.CreateDynamicBody(shapes=b2PolygonShape(
    vertices=module.tolist()), position=rand(polygon), angle=random.random() * math.pi), range(B_P))
for b in borders + modules:
    b.fixtures[0].sensor = True

for m in modules:
    m.bullet = True

In [6]:
def my_draw_polygon(self, body):
    pg.draw.lines(screen, (255, 0, 0, 0), True, map(lambda v: world2screen(body.transform * v), self.vertices))
    for c in filter(lambda c: c.contact.touching, body.contacts):
        vertices = [world2screen(v) for v in [body.position, c.other.position]]
        pg.draw.lines(screen, (0, 0, 255, 0), False, vertices)
        A = c.contact.fixtureA
        B = c.contact.fixtureB
        res = b2Distance(shapeA=A.shape, shapeB=B.shape,
                         transformA=A.body.transform, transformB=B.body.transform)

        def f2i(p): return map(int, world2screen(p))
        pg.draw.circle(screen, (0, 255, 0, 0), f2i(res.pointA), 5)
        pg.draw.circle(screen, (0, 255, 0, 0), f2i(res.pointB), 5)


def my_draw_edge(self, body):
    pg.draw.lines(screen, (0, 0, 0, 0), True, map(lambda v: world2screen(body.transform * v), self.vertices))


b2.polygonShape.draw = my_draw_polygon
b2.edgeShape.draw = my_draw_edge

In [7]:
def weight(surf_dist, dist): return surf_dist ** 4 + dist ** -4


def coeffient(surf_dist, dist): return (1 - surf_dist / dist) ** 2 - 1


def included_angle(*ls):
    return (np.arctan2(*ls[0]) - np.arctan2(*ls[1]) + math.pi / 2) % math.pi - math.pi / 2


CCW90 = np.array([[0, -1], [1, 0]])

In [8]:
NEIGHBOUR_FLAG = True
OVERLAP_FLAG = None

In [9]:
screen = pg.display.set_mode(SCREEN, 0, 32)
pg.display.set_caption('ITPLA')
clock = pg.time.Clock()

while not any(map(lambda e: e.type == QUIT or (e.type == KEYDOWN and e.key == K_ESCAPE), pg.event.get())):
    screen.fill((220, 220, 220, 0))
    for b in world.bodies:
        b.fixtures[0].shape.draw(b)
    pg.display.flip()
    clock.tick()  # TARGET_FPS)
    pg.display.set_caption('ITPLA {}'.format(clock.get_fps()))

    num = len(modules)
    position = map(lambda m: m.position, modules)
    vertices = map(body2worldvs, modules)
    distance = map(lambda vs: map(lambda p: map(
        lambda v: np.linalg.norm(p - v), vs), position), vertices)
    edge = map(lambda ds: map(lambda d: tuple(sorted(zip(
        *hq.nsmallest(2, zip(range(N), d), key=operator.itemgetter(-1)))[0])), ds), distance)
    neighbour = map(lambda _: collections.defaultdict(list), range(num))
    for i in range(num):
        for j in range(num):
            if i != j:
                neighbour[i][edge[i][j]].append(
                    (j, np.linalg.norm(position[i] - position[j])))
    neighbour = map(lambda i: dict(map(lambda e: [e, min(
        neighbour[i][e], key=operator.itemgetter(-1))[0]], neighbour[i])), range(num))

    for i in range(num):
        mi, pi, ivs = modules[i], position[i], vertices[i]
        ies = zip(np.roll(ivs, 2), ivs)

        weights, linearVelocity, angularVelocity = [], [], []

        for ei, j in neighbour[i].iteritems():
            mj, pj, jvs = modules[j], position[j], vertices[j]
            jes = zip(np.roll(jvs, 2), jvs)

            dist = np.linalg.norm(pi - pj)
            r = np.array(pi - pj)
            r /= np.linalg.norm(r)
            n = CCW90.dot(r)
            surf_dist = min(surface_distance(ivs, jes, n, r) +
                            surface_distance(jvs, ies, -n, r))
            k_r = coeffient(surf_dist, dist)

            ej = edge[j][i]
            eivs = operator.itemgetter(*ei)(vertices[i])
            ejvs = operator.itemgetter(*ej)(vertices[j])
            k_n = distance2line(np.mean(eivs, axis=0), ((0, 0), r))\
                - distance2line(np.mean(ejvs, axis=0), ((0, 0), r))
            if NEIGHBOUR_FLAG:
                weights.append(weight(surf_dist, dist))
                linearVelocity.append(k_r * r)
                linearVelocity[-1] -= k_n * n
                ia = included_angle(np.subtract(*eivs), np.subtract(*ejvs))
                angularVelocity.append(ia)

        for c in filter(lambda c: c.contact.touching, mi.contacts):
            ovs = body2worldvs(c.other)
            if isinstance(c.other.fixtures[0].shape, b2EdgeShape):
                dist = distance2line(pi, ovs)
                v = np.subtract(*ovs)
                v /= np.linalg.norm(v)
                n = CCW90.dot(v)
                if 0 < dist:
                    surf_dist = min(
                        [0] + surface_distance(ivs, [ovs], v, n) + surface_distance(ovs, ies, -v, n))
                    k_n = coeffient(surf_dist, dist)
                    if OVERLAP_FLAG:
                        weights.append(weight(surf_dist, dist))
                        linearVelocity.append(-k_n * n)
                        angularVelocity.append(included_angle(np.subtract(
                            *ies[np.argmin(map(lambda e: distance2line(np.mean(e, axis=0), ovs), ies))]), np.subtract(*ovs)))
            elif isinstance(c.other.fixtures[0].shape, b2PolygonShape):
                pass
        mi.linearVelocity *= lam
        mi.angularVelocity *= lam
        if weights:
            weights = np.array(weights) / sum(weights)
            mi.linearVelocity += np.average(linearVelocity,
                                            axis=0, weights=weights) / 10
            mi.angularVelocity += np.average(angularVelocity,
                                             axis=0, weights=weights) / 10

    world.Step(timeStep=TIME_STEP, velocityIterations=6, positionIterations=2)

pg.quit()